<a href="https://colab.research.google.com/github/alvinak2/green-watt/blob/main/GreenWatt_Energy_Solutions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [96]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import randint, uniform
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.inspection import permutation_importance

In [97]:
green_watt = pd.read_csv("/content/train.csv")
green_watt.head()

,timestamp,active_power_calculated_by_converter,active_power_raw,ambient_temperature,generator_speed,generator_winding_temp_max,grid_power10min_average,nc1_inside_temp,nacelle_temp,reactice_power_calculated_by_converter,reactive_power,wind_direction_raw,wind_speed_raw,wind_speed_turbulence,turbine_id,Target
0,2021-02-19 20:18:00,816.636759,834.917206,31.694380,1159.616602,65.954214,917.897085,31.881972,31.504713,141.457644,165.501518,280.864782,7.057000,0.544082,Turbine_108,47.582787
1,2021-04-27 04:55:00,419.107829,421.050873,12.894948,928.747996,59.571319,445.554250,32.423705,32.755770,89.186457,113.835236,299.552460,5.474937,0.469031,Turbine_18,46.070328
2,2021-01-25 06:26:00,1303.530558,1337.566142,16.648388,1201.219775,61.270498,1364.716003,11.446849,18.332985,230.622309,281.452253,84.960106,8.092457,0.622318,Turbine_105,39.989236
3,2021-10-30 03:47:00,61.494872,53.481008,28.388141,769.806122,40.674348,14.324897,34.253204,32.662889,66.211015,75.017531,87.261119,4.071032,0.760719,Turbine_15,46.056587
4,2021-03-15 00:39:00,593.514364,611.659108,31.519527,1046.916768,64.341763,599.020172,32.405586,31.466387,137.163938,160.202421,313.724818,6.357943,0.346068,Turbine_01,54.346095


In [98]:
green_watt.shape

(909604, 16)

In [99]:
green_watt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 909604 entries, 0 to 909603
Data columns (total 16 columns):
 #   Column                                  Non-Null Count   Dtype  
---  ------                                  --------------   -----  
 0   timestamp                               909604 non-null  object 
 1   active_power_calculated_by_converter    909604 non-null  float64
 2   active_power_raw                        909604 non-null  float64
 3   ambient_temperature                     909604 non-null  float64
 4   generator_speed                         909604 non-null  float64
 5   generator_winding_temp_max              909604 non-null  float64
 6   grid_power10min_average                 909604 non-null  float64
 7   nc1_inside_temp                         909604 non-null  float64
 8   nacelle_temp                            909604 non-null  float64
 9   reactice_power_calculated_by_converter  909604 non-null  float64
 10  reactive_power                          9096

In [100]:
green_watt.describe()

,active_power_calculated_by_converter,active_power_raw,ambient_temperature,generator_speed,generator_winding_temp_max,grid_power10min_average,nc1_inside_temp,nacelle_temp,reactice_power_calculated_by_converter,reactive_power,wind_direction_raw,wind_speed_raw,wind_speed_turbulence,Target
count,909604.000000,909604.000000,909604.000000,909604.000000,909604.000000,909604.000000,909604.000000,909604.000000,909604.000000,909604.000000,909604.000000,909604.000000,909604.000000,909604.000000
mean,525.886061,537.639468,25.573921,931.130882,60.730112,527.579537,31.755823,31.686413,111.080610,133.692333,199.487161,5.894950,0.651916,46.328595
std,474.619576,482.394907,8.025649,193.674384,9.721582,472.656976,3.929794,3.554426,78.219944,91.232152,104.617261,1.977684,0.317513,2.617691
min,0.000000,0.000008,5.616541,0.000000,25.636366,0.000000,11.141922,16.010218,-318.393721,-5.461607,0.309692,0.904500,0.000000,25.865320
25%,149.620898,155.813244,17.093680,770.538574,54.916767,147.181552,29.913803,30.267423,81.540850,96.045765,98.916060,4.400753,0.432129,44.816740
50%,379.989929,383.985926,27.919035,884.068502,60.034490,391.303187,31.762946,31.961610,92.005826,107.633378,232.821159,5.637207,0.583863,46.302899
75%,781.357475,800.261108,32.171696,1123.335856,65.737187,794.875710,33.469745,33.546260,124.879153,154.153730,287.667503,7.092771,0.798240,47.733105
max,1999.999858,1999.984456,48.089017,1267.140625,129.846405,1999.991455,52.790030,48.406106,523.892695,592.815938,359.455531,19.795001,9.534769,65.037689


In [101]:
green_watt.isnull().sum().sum()

np.int64(0)

In [102]:
green_watt.duplicated().sum()

np.int64(0)

In [103]:
green_watt['timestamp'] = pd.to_datetime(green_watt['timestamp'])
green_watt['hour'] = green_watt['timestamp'].dt.hour
green_watt['day_of_week'] = green_watt['timestamp'].dt.dayofweek
green_watt['month'] = green_watt['timestamp'].dt.month
green_watt['year'] = green_watt['timestamp'].dt.year

green_watt["turbine_id"] = green_watt["turbine_id"].astype("category")
green_watt["turbine_id_code"] = green_watt["turbine_id"].cat.codes

green_watt.head()

,timestamp,active_power_calculated_by_converter,active_power_raw,ambient_temperature,generator_speed,generator_winding_temp_max,grid_power10min_average,nc1_inside_temp,nacelle_temp,reactice_power_calculated_by_converter,...,wind_direction_raw,wind_speed_raw,wind_speed_turbulence,turbine_id,Target,hour,day_of_week,month,year,turbine_id_code
0,2021-02-19 20:18:00,816.636759,834.917206,31.694380,1159.616602,65.954214,917.897085,31.881972,31.504713,141.457644,...,280.864782,7.057000,0.544082,Turbine_108,47.582787,20,4,2,2021,4
1,2021-04-27 04:55:00,419.107829,421.050873,12.894948,928.747996,59.571319,445.554250,32.423705,32.755770,89.186457,...,299.552460,5.474937,0.469031,Turbine_18,46.070328,4,1,4,2021,12
2,2021-01-25 06:26:00,1303.530558,1337.566142,16.648388,1201.219775,61.270498,1364.716003,11.446849,18.332985,230.622309,...,84.960106,8.092457,0.622318,Turbine_105,39.989236,6,0,1,2021,3
3,2021-10-30 03:47:00,61.494872,53.481008,28.388141,769.806122,40.674348,14.324897,34.253204,32.662889,66.211015,...,87.261119,4.071032,0.760719,Turbine_15,46.056587,3,5,10,2021,10
4,2021-03-15 00:39:00,593.514364,611.659108,31.519527,1046.916768,64.341763,599.020172,32.405586,31.466387,137.163938,...,313.724818,6.357943,0.346068,Turbine_01,54.346095,0,0,3,2021,0


In [104]:
TARGET = None
for c in ["target","Target","power","power_output","Power","power_kw","output"]:
    if c in green_watt.columns:
        TARGET = c
        break
if TARGET is None:
    raise ValueError("Please set TARGET to your power column name in green_watt.")

In [105]:
df = green_watt.copy()

In [106]:
if "wind_speed" in df.columns:
    df["wind_speed_cubed"] = df["wind_speed"] ** 3

In [107]:
sensors = [c for c in ["wind_speed","generator_speed", TARGET, "ambient_temp"] if c in df.columns]
for s in sensors:
    df[f"{s}_lag1"] = df.groupby("turbine_id_code")[s].shift(1)
    df[f"{s}_roll3_mean"] = df.groupby("turbine_id_code")[s].shift(1).rolling(3, min_periods=1).mean().reset_index(level=0, drop=True)

lag_cols = [c for c in df.columns if ("_lag1" in c or "_roll3_mean" in c)]
df = df.dropna(subset=lag_cols).reset_index(drop=True)

In [108]:
to_drop = [c for c in df.columns if df[c].dtype == "object" and c not in ("turbine_id_code",)]
df = df.drop(columns=to_drop)

In [109]:
def per_turbine_split(df, tid_col='turbine_id_code', time_col='timestamp', val_frac=0.1, test_frac=0.1):
    if time_col in df.columns:
        df = df.sort_values([tid_col, time_col]).reset_index(drop=True)
    else:
        df = df.sort_values([tid_col]).reset_index(drop=True)
    train_idx, val_idx, test_idx = [], [], []
    for _, g in df.groupby(tid_col):
        n = len(g)
        t0 = int(n * (1 - test_frac))
        v0 = int(n * (1 - test_frac - val_frac))
        idx = g.index.values
        train_idx += idx[:v0].tolist()
        val_idx += idx[v0:t0].tolist()
        test_idx += idx[t0:].tolist()
    return df.loc[train_idx].reset_index(drop=True), df.loc[val_idx].reset_index(drop=True), df.loc[test_idx].reset_index(drop=True)

train_df, val_df, test_df = per_turbine_split(df)

In [112]:
FEATURES = [c for c in df.columns if c != TARGET and c != "timestamp" and c != "turbine_id"]
X_train, y_train = train_df[FEATURES], train_df[TARGET]
X_val, y_val     = val_df[FEATURES], val_df[TARGET]
X_test, y_test   = test_df[FEATURES], test_df[TARGET]

print("Train/Val/Test sizes:", X_train.shape[0], X_val.shape[0], X_test.shape[0])

Train/Val/Test sizes: 727665 90957 90966


In [ ]:
rf = RandomForestRegressor(random_state=42, n_jobs=-1)

param_dist = {
    "n_estimators": randint(100, 400),
    "max_depth": randint(6, 30),
    "min_samples_split": randint(2, 8),
    "min_samples_leaf": randint(1, 6),
    "max_features": ["sqrt", "log2", None],
}

search = RandomizedSearchCV(
    rf,
    param_distributions=param_dist,
    n_iter=20,            # quick but effective; increase to 50-100 if you have time
    scoring="neg_root_mean_squared_error",
    cv=3,
    verbose=1,
    n_jobs=-1,
    random_state=42
)

print("Running RandomizedSearchCV (RandomForest)...")
search.fit(X_train, y_train)

best_rf = search.best_estimator_
print("Best params:", search.best_params_)

Running RandomizedSearchCV (RandomForest)...
Fitting 3 folds for each of 20 candidates, totalling 60 fits


In [ ]:
def metrics(y_true, y_pred):
    return {
        "rmse": mean_squared_error(y_true, y_pred, squared=False),
        "mae": mean_absolute_error(y_true, y_pred),
        "r2": r2_score(y_true, y_pred)
    }

val_preds = best_rf.predict(X_val)
test_preds = best_rf.predict(X_test)

print("Validation metrics:", metrics(y_val, val_preds))
print("Test metrics:", metrics(y_test, test_preds))

In [ ]:
print("Computing permutation importances (may take time)...")
perm = permutation_importance(best_rf, X_val, y_val, n_repeats=10, random_state=42, n_jobs=-1)
feat_imp = pd.Series(perm.importances_mean, index=FEATURES).sort_values(ascending=False)
print("Top features:", feat_imp.head(10))

# plot top 10 feature importances
feat_imp.head(10).plot.barh(figsize=(8,5))
plt.gca().invert_yaxis()
plt.title("Permutation feature importance (validation)")
plt.show()

In [ ]:
resid_df = test_df.copy().reset_index(drop=True)
resid_df["pred"] = test_preds
resid_df["residual"] = resid_df[TARGET] - resid_df["pred"]

per_turb_res = resid_df.groupby("turbine_id_code")["residual"].agg(["mean","std"]).reset_index().rename(columns={"mean":"resid_mean","std":"resid_std"})
resid_df = resid_df.merge(per_turb_res, on="turbine_id_code", how="left")
resid_df["resid_zscore"] = (resid_df["residual"] - resid_df["resid_mean"]) / (resid_df["resid_std"] + 1e-9)
resid_df["anomaly"] = resid_df["resid_zscore"].abs() > 3.0
n_anom = resid_df["anomaly"].sum()
print(f"Anomalies flagged in test set (|z|>3): {n_anom}")

In [ ]:
resid_df.loc[resid_df["anomaly"], ["turbine_id_code", "pred", TARGET, "residual", "resid_zscore"]].to_csv("anomaly_report.csv", index=False)
joblib.dump(best_rf, "greenwatt_rf_model.joblib")
print("Saved model -> greenwatt_rf_model.joblib and anomaly_report.csv")